<a href="https://colab.research.google.com/github/denisecammarota/icgpu/blob/main/tpfinal/TimeCorr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Final Project

## GPU checking, setup of Rapids.ai

We check what GPU we have, just in case it's not supported by Rapids.ai, set up Rapids, install from GitHub. 

In [2]:
#checking what gpu google has given us
!nvidia-smi

Wed Jun  9 13:05:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os, shutil

sys.path.append('/usr/local/lib/python3.7/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ["CONDA_PREFIX"] = "/usr/local"
for so in ['cudf', 'rmm', 'nccl', 'cuml', 'cugraph', 'xgboost', 'cuspatial']:
  fn = 'lib'+so+'.so'
  source_fn = '/usr/local/lib/'+fn
  dest_fn = '/usr/lib/'+fn
  if os.path.exists(source_fn):
    print(f'Copying {source_fn} to {dest_fn}')
    shutil.copyfile(source_fn, dest_fn)
# fix for BlazingSQL import issue
# ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.26' not found (required by /usr/local/lib/python3.7/site-packages/../../libblazingsql-engine.so)
if not os.path.exists('/usr/lib64'):
    os.makedirs('/usr/lib64')
for so_file in os.listdir('/usr/local/lib'):
  if 'libstdc' in so_file:
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib64/'+so_file)
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib/x86_64-linux-gnu/'+so_file)

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 238 (delta 36), reused 11 (delta 5), pack-reused 171
Receiving objects: 100% (238/238), 73.89 KiB | 713.00 KiB/s, done.
Resolving deltas: 100% (98/98), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.19 and 0.20)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.19

	import sys, os, shutil

	sys.path.append('/usr/local/lib/python3.7/site-packages/')
	os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
	os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
	os.environ['CONDA_PREFIX'] = '/usr/local'
	for so in ['cudf', 'rmm'

## Auxiliary functions for data processing

Changing numpy for cupy, and pandas for cudf

In [4]:
import cudf
import cupy as cp

In [5]:
import matplotlib.pyplot as plt
import numpy as np

In [6]:
def change_dates(df):
    #changes fecha_inicio_sintomas according to new criteria
    filt_df2 = (df.fecha_inicio_sintomas.isnull()) #filter fecha_inicio_sintomas = inexistant
    n_size = (df[filt_df2]).shape[0]
    df["fecha_inicio_sintomas"] = df["fecha_inicio_sintomas"].fillna(0)
    df["fecha_inicio_sintomas"] = df["fecha_inicio_sintomas"].astype('int')
    df["fecha_apertura"] = df["fecha_apertura"].astype('int')
    b = cp.array(df["fecha_apertura"].values)
    a = cp.array(df["fecha_inicio_sintomas"].values)
    a[filt_df2.values] = b[filt_df2.values] - cp.random.randint(0,9,a[filt_df2.values].shape)
    filt_a = cp.where(a<0)
    a[filt_a] = 0
    df = df.drop(["fecha_inicio_sintomas"],axis=1)
    df["fecha_inicio_sintomas"] = a
    return df

## Data analysis per se, we get data and process 

For this part, don't forget to upload the data from pc to the google drive of the user. First, I authorize colab to get data from my drive for this, so that I don't have to upload to the colab session all the time. We always have to authorize the drive, tho. 

In [7]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
dir = '/content/gdrive/My Drive/Datos_Covid/Covid19Casos.csv'

In [9]:
data = cudf.read_csv(dir,sep=",",quotechar='"',
                   parse_dates=["fecha_inicio_sintomas","fecha_apertura"],na_values=['']) 
data = data[data["clasificacion_resumen"] == "Confirmado"] #filter confirmed cases
df = cudf.DataFrame(data) 

In [10]:
df = df.drop(['clasificacion_resumen'], axis=1) #drop clasificacion_resumen bc previous filter of confirmed cases

In [11]:
filt_df1 = (df.fecha_inicio_sintomas.notnull()) #filter fecha_inicio_sintomas = inexistant
inicio_epidemia = (df.loc[filt_df1,"fecha_inicio_sintomas"]).min() #first symptoms of a person registered
ultima_actualizacion_sintomas = (df.loc[filt_df1,"fecha_inicio_sintomas"]).max() #last day symptoms of a person registered
ultima_actualizacion_apertura = (df["fecha_apertura"]).max()
df["fecha_inicio_sintomas"] -= inicio_epidemia #correcting by inicio_epidemia 
df["fecha_apertura"] -= inicio_epidemia #correcting by inicio_epidemia
df.fecha_inicio_sintomas = df.fecha_inicio_sintomas.dt.days #change to int, ditch days 
df.fecha_apertura = df.fecha_apertura.dt.days #change to int, ditch days
print("primer sintoma de persona confirmada: ",inicio_epidemia)
print("ultimo sintoma de persona confirmada: ",ultima_actualizacion_sintomas)
print("ultima apertura de persona confirmada: ",ultima_actualizacion_apertura)

primer sintoma de persona confirmada:  2020-01-01T00:00:00.000000000
ultimo sintoma de persona confirmada:  2021-05-28T00:00:00.000000000
ultima apertura de persona confirmada:  2021-05-28T00:00:00.000000000


In [12]:
df = change_dates(df) # we have replaced all bad or undefined dates

In [13]:
df

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion,fecha_inicio_sintomas
16,10000015,F,48,Años,Argentina,Entre Ríos,Nogoyá,Entre Ríos,463,14,<NA>,NO,<NA>,NO,<NA>,NO,30,Público,Caso confirmado por laboratorio - No activo (p...,30,2021-04-08,077,2021-05-28,458
27,10000026,F,23,Años,Argentina,Córdoba,Capital,Córdoba,463,14,<NA>,NO,<NA>,NO,<NA>,NO,14,Público,Caso confirmado por laboratorio - No activo (p...,14,2021-04-05,014,2021-05-28,458
51,10000049,F,73,Años,Argentina,CABA,COMUNA 09,CABA,463,14,<NA>,NO,<NA>,NO,<NA>,NO,02,Público,Caso confirmado por laboratorio - No activo (p...,02,2021-04-10,009,2021-05-28,461
58,10000055,F,42,Años,Argentina,CABA,COMUNA 09,CABA,463,14,<NA>,NO,<NA>,NO,<NA>,NO,02,Público,Caso confirmado por laboratorio - No activo (p...,02,2021-04-14,009,2021-05-28,461
60,10000057,M,35,Años,Argentina,Buenos Aires,Mercedes,Buenos Aires,463,14,<NA>,NO,<NA>,NO,<NA>,NO,06,Público,Caso confirmado por laboratorio - No activo (p...,06,2021-04-06,532,2021-05-28,457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11813627,9999973,M,36,Años,Argentina,Buenos Aires,San Isidro,Buenos Aires,463,14,<NA>,NO,<NA>,NO,<NA>,NO,06,Privado,Caso confirmado por laboratorio - No activo (p...,06,2021-04-08,756,2021-05-28,461
11813634,9999980,F,17,Años,Argentina,Buenos Aires,San Isidro,Buenos Aires,463,14,<NA>,NO,<NA>,NO,<NA>,NO,06,Privado,Caso confirmado por laboratorio - No activo (p...,06,2021-04-09,756,2021-05-28,458
11813640,9999986,M,17,Años,Argentina,Santa Cruz,Güer Aike,Santa Cruz,463,14,<NA>,NO,<NA>,NO,<NA>,NO,78,Público,Caso confirmado por laboratorio - No activo (p...,78,2021-04-09,021,2021-05-28,460
11813647,9999992,F,34,Años,Argentina,Buenos Aires,Merlo,Buenos Aires,463,14,<NA>,NO,<NA>,NO,<NA>,NO,06,Público,Caso confirmado por laboratorio - No activo (p...,06,2021-04-09,539,2021-05-28,459


In [14]:
#let's list all the provinces that have cases (all rn tbh, but just to use pandas stuff)
#we'll get those times series then, with cudf grouping
provincias = df["carga_provincia_nombre"].unique()
#print: we get provinces + the sin especificar / unespecified thing
print(provincias)

0            Buenos Aires
1                    CABA
2               Catamarca
3                   Chaco
4                  Chubut
5              Corrientes
6                 Córdoba
7              Entre Ríos
8                 Formosa
9                   Jujuy
10               La Pampa
11               La Rioja
12                Mendoza
13               Misiones
14                Neuquén
15              Río Negro
16                  Salta
17               San Juan
18               San Luis
19             Santa Cruz
20               Santa Fe
21    Santiago del Estero
22       Tierra del Fuego
23                Tucumán
Name: carga_provincia_nombre, dtype: object


## Getting the time series themselves

Now we'll figure out how to get the time series thing working

In [15]:
df = df[["carga_provincia_nombre","fecha_inicio_sintomas"]]

In [16]:
df2 = df.groupby(["carga_provincia_nombre","fecha_inicio_sintomas"]).size().reset_index() 

In [17]:
df2 = df2.rename(columns={0:'casos'})

In [18]:
df3 = df2.sort_values(["carga_provincia_nombre","fecha_inicio_sintomas"],ascending=[True,True])

/usr/local/lib/python3.7/site-packages/cudf/core/frame.py:2582: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  "When using a sequence of booleans for `ascending`, "


In [19]:
rows = provincias.shape[0]
cols = df2["fecha_inicio_sintomas"].max()
time_series = cp.zeros(shape=(rows,cols))
time_series_ac = cp.zeros(shape=(rows,cols))

In [20]:
j = 0
for i in provincias.to_pandas():
  df_tmp = df3[df3["carga_provincia_nombre"]==i]
  tmp_dates = cp.array(df_tmp["fecha_inicio_sintomas"].values)
  tmp_cases = cp.array(df_tmp["casos"].values)
  time_series[j,tmp_dates]= tmp_cases
  j = j + 1

In [21]:
for i in range(rows):
  for j in range(13,cols):
    time_series_ac[i,j] = cp.sum(time_series[i,j-13:j+1])

In [22]:
time_series_ac

array([[     0.,      0.,      0., ..., 110052., 104271.,  97198.],
       [     0.,      0.,      0., ...,  47856.,  44736.,  41200.],
       [     0.,      0.,      0., ...,   3901.,   3714.,   3484.],
       ...,
       [     0.,      0.,      0., ...,   6694.,   6429.,   6073.],
       [     0.,      0.,      0., ...,    715.,    678.,    622.],
       [     0.,      0.,      0., ...,  11335.,  11087.,  10532.]])

## Correlation finding techniques

All adapted from: https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9

### Pearson R programmed in CuPy

In [23]:
#no existe en cupy, la hago yo a esta funcion 
def pearsonr(a,b):
  m_a = cp.mean(a)
  m_b = cp.mean(b)
  r_a = a - m_a
  r_b = b - m_b
  r_a_2 = cp.sum((r_a)**2) 
  r_b_2 = cp.sum((r_b)**2)
  r = cp.sum((r_a*r_b)/cp.sqrt(r_a_2*r_b_2))
  return r

In [24]:
#por ejemplo, correlaciones para caba y baires
caba = time_series_ac[1,:-10]
baires = time_series_ac[0,:-10]
pearsonr(caba,baires)

array(0.97606147)

In [25]:
#ahora, tratemos de hacer lo mismo para baires y el resto del país
baires = time_series_ac[0,:-10]
i = 1
provs = provincias.to_pandas()
print('Coeficientes de Pearson')
while i < rows:
  prov_tmp = time_series_ac[i,:-10]
  r = pearsonr(baires,prov_tmp)
  print('Pearson entre '+provs[i]+' y Baires: ',cp.around(r,3))
  i += 1

Coeficientes de Pearson
Pearson entre CABA y Baires:  0.976
Pearson entre Catamarca y Baires:  0.739
Pearson entre Chaco y Baires:  0.756
Pearson entre Chubut y Baires:  0.415
Pearson entre Corrientes y Baires:  0.618
Pearson entre Córdoba y Baires:  0.79
Pearson entre Entre Ríos y Baires:  0.793
Pearson entre Formosa y Baires:  0.443
Pearson entre Jujuy y Baires:  0.575
Pearson entre La Pampa y Baires:  0.677
Pearson entre La Rioja y Baires:  0.765
Pearson entre Mendoza y Baires:  0.859
Pearson entre Misiones y Baires:  0.56
Pearson entre Neuquén y Baires:  0.581
Pearson entre Río Negro y Baires:  0.655
Pearson entre Salta y Baires:  0.753
Pearson entre San Juan y Baires:  0.7
Pearson entre San Luis y Baires:  0.749
Pearson entre Santa Cruz y Baires:  0.554
Pearson entre Santa Fe y Baires:  0.712
Pearson entre Santiago del Estero y Baires:  0.697
Pearson entre Tierra del Fuego y Baires:  0.46
Pearson entre Tucumán y Baires:  0.642


### Time Lagged Cross Correlation

In [26]:
def lag_pearson(a,b,lag):
  #dejo a fijo
  #desplazo b
  b = cp.roll(b,lag)
  r = pearsonr(a[lag:],b[lag:])
  return r

#### Primer intento (40 fps, 5 seconds)

In [51]:
#ahora, tratemos de hacer lo mismo para baires y el resto del país
baires = time_series_ac[0,:-10]
offset_vec = []
rs_vec = []
seconds = 5
fps = 40
i = 1
while i < rows:
  prov_tmp = time_series_ac[i,:-10]
  rs = [lag_pearson(baires,prov_tmp,lag) for lag in range(-int(seconds*fps),int(seconds*fps+1))]
  rs = cp.array(rs)
  #print(rs)
  offset_tmp = cp.floor(len(rs)/2)-cp.argmax(rs)
  rs_tmp = cp.max(rs)
  offset_vec.append(offset_tmp)
  rs_vec.append(rs_tmp)
  print('Pearson máx entre '+provs[i]+' y Baires: ',cp.around(rs_tmp,3),'. En offset de frames: ',offset_tmp)
  i = i + 1

Pearson máx entre CABA y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Catamarca y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Chaco y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Chubut y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Corrientes y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Córdoba y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Entre Ríos y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Formosa y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Jujuy y Baires:  nan . En offset de frames:  8.0
Pearson máx entre La Pampa y Baires:  nan . En offset de frames:  8.0
Pearson máx entre La Rioja y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Mendoza y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Misiones y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Neuquén y Baires:  nan . En offset de frames:  8.0
Pearson máx entre Río Negro y B

In [54]:
#imprimamos todo a ver qué onda, que es lo que pasa?
baires = time_series_ac[0,:-10]
offset_vec = []
rs_vec = []
seconds = 5
fps = 40
i = 1
while i < rows:
  prov_tmp = time_series_ac[i,:-10]
  rs = [lag_pearson(baires,prov_tmp,lag) for lag in range(-int(seconds*fps),int(seconds*fps+1))]
  rs = cp.array(rs)
  print(rs)
  offset_tmp = cp.floor(len(rs)/2)-cp.argmax(rs)
  rs_tmp = cp.max(rs)
  offset_vec.append(offset_tmp)
  rs_vec.append(rs_tmp)
  print('Pearson máx entre '+provs[i]+' y Baires: ',cp.around(rs_tmp,3),'. En offset de frames: ',offset_tmp)
  i = i + 1

[ 0.8834627   0.88098672  0.87825489  0.87536196  0.87236045  0.86927848
  0.86607544  0.86277436  0.85938984  0.85599219  0.85266172  0.84929376
  0.84605422  0.84285354  0.83986076  0.8370071   0.83435382  0.83173654
  0.82924901  0.82677338  0.82451888  0.82229419  0.82019234  0.81812177
  0.81604992  0.81395105  0.8116082   0.80903902  0.80597525  0.80252243
  0.79856695  0.79389704  0.78844969  0.78210534  0.77471685  0.7668354
  0.75832556  0.74924285  0.73947263  0.72942437  0.71909507  0.70859871
  0.69799158  0.68729017  0.6766616   0.66624117  0.65605117  0.64657842
  0.63761256  0.62910204  0.62155806  0.61502428  0.60969296  0.60547108
  0.60242117  0.60057115  0.59960773  0.60065143  0.6030626   0.60743576
  0.61414443  0.62194183  0.63240636  0.64346345  0.65663908  0.67114613
  0.68681286  0.70420744  0.72211461  0.74003453  0.75853189  0.77583448
  0.7927509   0.80827974  0.82227309  0.83570787  0.84714658  0.85843147
  0.86749665  0.87551722  0.88023539  0.88224664  0.

#### Segundo intento (20 fps, 5 seconds)

In [58]:
#ahora, tratemos de hacer lo mismo para baires y el resto del país
baires = time_series_ac[0,:-10]
offset_vec = []
rs_vec = []
seconds = 5
fps = 20
i = 1
while i < rows:
  prov_tmp = time_series_ac[i,:-10]
  rs = [lag_pearson(baires,prov_tmp,lag) for lag in range(-int(seconds*fps),int(seconds*fps+1))]
  rs = cp.array(rs)
  #print(rs)
  offset_tmp = cp.floor(len(rs)/2)-cp.argmax(rs)
  rs_tmp = cp.max(rs)
  offset_vec.append(offset_tmp)
  rs_vec.append(rs_tmp)
  print('Pearson máx entre '+provs[i]+' y Baires: ',cp.around(rs_tmp,3),'. En offset de frames: ',offset_tmp)
  i = i + 1

Pearson máx entre CABA y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Catamarca y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Chaco y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Chubut y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Corrientes y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Córdoba y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Entre Ríos y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Formosa y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Jujuy y Baires:  nan . En offset de frames:  4.0
Pearson máx entre La Pampa y Baires:  nan . En offset de frames:  4.0
Pearson máx entre La Rioja y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Mendoza y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Misiones y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Neuquén y Baires:  nan . En offset de frames:  4.0
Pearson máx entre Río Negro y B

In [59]:
#imprimamos todo a ver qué onda, que es lo que pasa?
baires = time_series_ac[0,:-10]
offset_vec = []
rs_vec = []
seconds = 5
fps = 20
i = 1
while i < rows:
  prov_tmp = time_series_ac[i,:-10]
  rs = [lag_pearson(baires,prov_tmp,lag) for lag in range(-int(seconds*fps),int(seconds*fps+1))]
  rs = cp.array(rs)
  print(rs)
  offset_tmp = cp.floor(len(rs)/2)-cp.argmax(rs)
  rs_tmp = cp.max(rs)
  offset_vec.append(offset_tmp)
  rs_vec.append(rs_tmp)
  print('Pearson máx entre '+provs[i]+' y Baires: ',cp.around(rs_tmp,3),'. En offset de frames: ',offset_tmp)
  i = i + 1

[ 0.57059784  0.54662619  0.5232292   0.50061641  0.47865147  0.45764704
  0.43740518  0.41793352  0.39937543  0.38164931  0.3644669   0.34806355
  0.33215709  0.31675432  0.30166832  0.28709097  0.27292328  0.25872841
  0.24471587  0.22976364  0.2143114   0.19766112  0.1790515   0.15857636
  0.13613443  0.11286876  0.08890024  0.06398786  0.04079559  0.01882591
  0.00313349 -0.00583495 -0.0136949  -0.01502963 -0.01452793 -0.01472018
 -0.01069118 -0.0072226  -0.00214152  0.00262134  0.01615936  0.02634372
  0.03650828  0.05661966  0.07679986  0.09285949  0.10626955  0.11876941
  0.13148354  0.15552433  0.16488729  0.18514508  0.19907347  0.21369771
  0.20714658  0.2107919   0.20355056  0.18022772  0.15840174  0.14034659
  0.10881324  0.07847531  0.04837408  0.0042015  -0.01097412 -0.03678999
 -0.0566054  -0.08031274 -0.10838749 -0.13002801 -0.13529166 -0.15858898
 -0.18166062 -0.22591854 -0.2391673  -0.25608148 -0.26473449 -0.2706629
 -0.2662052  -0.22509298 -0.18692591 -0.10933261  0.